<a href="https://colab.research.google.com/github/WilyLynx/Piekna_corona_python/blob/master/Part_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DASH
czyli o tym jak stworzyć prostą aplikację do wizualizacji.

Dash jest freamworkiem pozwalającym na stworzenie w krótkim czasie prostych aplikacji analitycznych.

DEMO:

![Alt Text](https://images.prismic.io/plotly-marketing-website/db073c29-5cee-4f13-9adf-46dd136f45f4_ezgif.com-video-to-gif+%2811%29.gif?auto=compress,format)

Źródło: [https://plotly.com/dash/](https://plotly.com/dash/)

## Pierwsza aplikacja w Dash'u

UWAGA!!!

poniższe programy nie zadziałają bezpośrednio z Google Colab. Poniższy kod trzeba zapisać w formie pliku app.py i uruchomić z IDE lub command line poleceniem python app.py

Trzeba mieć też zainstalowane pakiety dash, dash_core_components oraz dash_html_components.

In [0]:
import dash
import dash_core_components as dcc
import dash_html_components as html

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='example-graph',
        figure={
            'data': [
                {'x': [1, 2, 3], 'y': [4, 1, 2], 'type': 'bar', 'name': 'SF'},
                {'x': [1, 2, 3], 'y': [2, 4, 5], 'type': 'bar', 'name': u'Montréal'},
            ],
            'layout': {
                'title': 'Dash Data Visualization'
            }
        }
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

Dash pozwala na budowanie strony internetowej przy pomocy znaczników HTML. Najciekawszym elementem jest obiekt dcc.Graph, w którym można umieszczać wykresy stworzone w Plotly.

## Plotly
Plotly jest biblioteką umożliwiającą stworzenie interaktywnych wykresów w Pythonie. Link do dokumentacji: [https://plotly.com/python/](https://plotly.com/python/)

In [0]:
import plotly.graph_objects as go
fig = go.Figure(
    data=[go.Bar(y=[2, 2, 3])],
    layout_title_text="A Figure Displayed with fig.show()"
)
fig.show()

### Wykresy dla danych z DataFrame

In [0]:
import pandas as pd
import numpy as np
import json
import requests

In [0]:
def read_grzegorz_files(url):
  raw_json = requests.request("GET", url).content
  data = json.loads(raw_json)['data']
  df =  pd.DataFrame(data, columns=['Data', 'Kwarantanna', 'Hospitalizowani', 'Potwierdzone przypadki', 'Ozdrowieńcy', 'Zgony'])
  df.loc[:, 'Data'] = pd.to_datetime(df['Data'],format='%d.%m.%Y')
  return df


url = "https://raw.githubusercontent.com/WilyLynx/Piekna_corona_python/master/data/minsk-psse.json"
minsk_df = read_grzegorz_files(url)
minsk_df.head()

In [0]:
import plotly.graph_objects as go

fig = go.Figure(go.Scatter(
    x = minsk_df['Data'],
    y = minsk_df['Kwarantanna']
))
fig.show()

In [0]:
import plotly.graph_objects as go

fig = go.Figure(go.Scatter(
    x = minsk_df['Data'],
    y = minsk_df['Kwarantanna']
))
fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(
    title="Liczba osób poddanych kwarantannie"
)
fig.show()

In [0]:
url = "https://raw.githubusercontent.com/WilyLynx/Piekna_corona_python/master/data/siedlce-psse.json"
siedlce_df = read_grzegorz_files(url)
siedlce_df.head()

In [0]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x = minsk_df['Data'],
    y = minsk_df['Kwarantanna'],
    name='Mińsk Mazowiecki'
))
fig.add_trace(go.Scatter(
    x = siedlce_df['Data'],
    y = siedlce_df['Kwarantanna'],
    name='Siedlce'
))
fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(
    title="Liczba osób poddanych kwarantannie"
)
fig.show()

In [0]:
minsk_df['Powiat'] = ['miński' for i in range(len(minsk_df))]
siedlce_df['Powiat'] = ['siedlecki' for i in range(len(siedlce_df))]
covid_df = pd.concat([minsk_df, siedlce_df], axis=0)
covid_df['Ozdrowieńcy'].fillna(value=0, inplace=True)
covid_df['Zgony'].fillna(value=0, inplace=True)
covid_df['Aktywne'] = covid_df['Potwierdzone przypadki'] - covid_df['Ozdrowieńcy'] - covid_df['Zgony']
covid_df.tail()

In [0]:
# Express 
import plotly.express as px
fig = px.line(covid_df, x='Data', y='Aktywne', color='Powiat')
fig.show()